### 데이터분석 필수 과제 
- 새로운 데이터를 공유하면
- 정답데이터만 알려드릴테니 우리가 지금까지 배운 내용을 이용해서 머신러닝을 통해 정확도를 높여주세요!
- 핵심적인 전처리와 데이터 특성에 대한 인사이트도 꼭 정리하셔야 합니다.
- 사전에 모든 정확도를 받아 볼 예정입니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("sales_data.csv", encoding="unicode_escape")

In [3]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


### 데이터 전처리

- 먼저 결측치 확인 후 대체부터 한다.
- 찾을 제품(y값) : HAND WARMER
- 새롭게 생성할 열은 y값에 의존하지 않도록 함

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# 결측치 확인
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

- Description 변수를 y로 두기 때문에 결측치인 행은 분석에서 제외
- CustomerID는 고객 고유 ID이고, 결측치 수가 많기 때문에 해당 열을 전부 제거

In [6]:
# CustomerID 컬럼 제거
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
df = df[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
         'UnitPrice', 'Country']]

In [8]:
# Description이 NaN인 행 제거
df = df.dropna()
df.reset_index(inplace=True)

### 상품명 관련
- 상품 종류를 구분하는 키워드를 중심으로 컬럼을 생성함
- packed : 'pack of' 포함 1, o.w. 0
- numbered : 숫자 포함 1, o.w. 0
- set : 'set of' 포함 1, o.w. 0
- box : 'box of' 포함 1, o.w. 0
- amp : '&' 포함 1, o.w. 0
- plus : '+' 포함 1, o.w. 0

In [10]:
import re

# Packed 변수 추가
df["temp"] = df["Description"].str.extract("(PACK OF)")

In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Packed"])
temp["Packed"].replace({0:1, 1:0}, inplace=True)
df["Packed"] = temp["Packed"]

In [13]:
# Numbered 변수 추가
df["temp"] = df["Description"].str.extract("([0-9]+)")
df["temp"] = df["temp"].astype("float")
df["temp"][df["temp"] >= 0] = 1

C:\Users\doram\AppData\Local\Temp\ipykernel_21740\1250047160.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp"][df["temp"] >= 0] = 1


In [14]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Numbered"])
temp["Numbered"].replace({0:1, 1:0}, inplace=True)
df["Numbered"] = temp["Numbered"]

In [15]:
# Set 변수 추가
df["temp"] = df["Description"].str.extract("(SET OF)")

In [16]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Set"])
temp["Set"].replace({0:1, 1:0}, inplace=True)
df["Set"] = temp["Set"]

In [17]:
# Box 변수 추가
df["temp"] = df["Description"].str.extract("(BOX OF)")

In [18]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Box"])
temp["Box"].replace({0:1, 1:0}, inplace=True)
df["Box"] = temp["Box"]

In [19]:
# Amp 변수 추가
df["temp"] = df["Description"].str.extract("(&)")

In [20]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Amp"])
temp["Amp"].replace({0:1, 1:0}, inplace=True)
df["Amp"] = temp["Amp"]

In [21]:
# Plus 변수 추가
df["temp"] = df["Description"].str.extract("(\+)")

In [22]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Plus"])
temp["Plus"].replace({0:1, 1:0}, inplace=True)
df["Plus"] = temp["Plus"]

### 상품 옵션 관련
- col_op : color option 포함 1, o.w. 0
> color options
> - WHITE, RED, GREEN, PINK, BLUE, ORANGE, YELLOW, PURPLE, BLACK, GREY, IVORY, BROWN
> - CHOC(CHOCOLATE), GOLD, SILVER, MINT
- des_op : 'design' 포함 1, o.w. 0

In [23]:
# Col_op 변수 추가
df["temp"] = df["Description"].str.extract("(WHITE|RED|GREEN|PINK|BLUE|ORANGE|YELLOW|PURPLE|BLACK|GREY|IVORY|BROWN|CHOC|GOLD|SILVER|MINT)")
df["temp"][~df["temp"].isna()] = 1

C:\Users\doram\AppData\Local\Temp\ipykernel_21740\917505287.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp"][~df["temp"].isna()] = 1


In [24]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Col_op"])
temp["Col_op"].replace({0:1, 1:0}, inplace=True)
df["Col_op"] = temp["Col_op"]

In [25]:
# Des_op 변수 추가
df["temp"] = df["Description"].str.extract("(DESIGN)")

In [26]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Des_op"])
temp["Des_op"].replace({0:1, 1:0}, inplace=True)
df["Des_op"] = temp["Des_op"]

### 상품 가격 관련
- hprice : price가 일정 수준(5) 이상일 때 1, o.w. 0
- lprice : price가 일정 수준(1) 미만일 때 (음수 제외) 1, o.w. 0

In [27]:
# Hprice 변수 추가
strprice = df["UnitPrice"].astype("str")
df["temp"] = strprice.str.extract("([5-9]\.|[1-9]\d+\.)")
df["temp"] = df["temp"].astype("float")
df["temp"][~df["temp"].isna()] = 1

C:\Users\doram\AppData\Local\Temp\ipykernel_21740\3481998269.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp"][~df["temp"].isna()] = 1


In [28]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Hprice"])
temp["Hprice"].replace({0:1, 1:0}, inplace=True)
df["Hprice"] = temp["Hprice"]

In [29]:
# Lprice 변수 추가
strprice = df["UnitPrice"].astype("str")
df["temp"] = strprice.str.extract("(0\.)")
df["temp"] = df["temp"].astype("float")
df["temp"][~df["temp"].isna()] = 1

C:\Users\doram\AppData\Local\Temp\ipykernel_21740\1926963694.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp"][~df["temp"].isna()] = 1


In [30]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Lprice"])
temp["Lprice"].replace({0:1, 1:0}, inplace=True)
df["Lprice"] = temp["Lprice"]

### 상품 개수 관련
- hquant : quantity가 일정 수준(10) 이상일 때 1, o.w. 0
- removed : quantity가 음수일 때 1, o.w. 0

In [31]:
# Hquant 변수 추가
strquant = df["Quantity"].astype("str")
df["temp"] = strquant.str.extract("([1-9]\d+)")
df["temp"] = df["temp"].astype("float")
df["temp"][~df["temp"].isna()] = 1

C:\Users\doram\AppData\Local\Temp\ipykernel_21740\3710890574.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp"][~df["temp"].isna()] = 1


In [32]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Hquant"])
temp["Hquant"].replace({0:1, 1:0}, inplace=True)
df["Hquant"] = temp["Hquant"]

In [33]:
# Removed 변수 추가
strquant = df["Quantity"].astype("str")
df["temp"] = strquant.str.extract("(\-\d+)")
df["temp"] = df["temp"].astype("float")
df["temp"][~df["temp"].isna()] = 1

C:\Users\doram\AppData\Local\Temp\ipykernel_21740\1120017883.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp"][~df["temp"].isna()] = 1


In [34]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Removed"])
temp["Removed"].replace({0:1, 1:0}, inplace=True)
df["Removed"] = temp["Removed"]

### 시계열 데이터 관련
- quarter : 분기(Q1~4)에 따라 원-핫 인코딩
- year : 연도별 원-핫 인코딩
- am : 오전이면 1, o.w. 0

In [35]:
# Quarter 변수 추가 후 원-핫 인코딩
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["Quarter"] = df["InvoiceDate"].dt.quarter
df["Quarter"].replace({1:"Q1", 2:"Q2", 3:"Q3", 4:"Q4"}, inplace=True)

oh_q = pd.get_dummies(df["Quarter"])
df = pd.concat([df, oh_q], axis=1)
df.drop(columns=["Quarter"], inplace=True)

In [36]:
# Year 변수 추가 후 원-핫 인코딩
df["temp"] = df["InvoiceDate"].dt.year
oh_y = pd.get_dummies(df["temp"])
df = pd.concat([df, oh_y], axis=1)

In [44]:
# Am 변수 추가
df["temp"] = df["InvoiceDate"].dt.hour - 12
df["temp"][df["temp"] >= 0] = np.nan
df["temp"][df["temp"] < 0] = 1

C:\Users\doram\AppData\Local\Temp\ipykernel_21740\2516896084.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temp"][df["temp"] >= 0] = np.nan


In [46]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Am"])
temp["Am"].replace({0:1, 1:0}, inplace=True)
df["Am"] = temp["Am"]

### 국가 관련
- country : 국가에 따라 원-핫 인코딩

In [58]:
# Country 변수로 원-핫 인코딩
oh_c = pd.get_dummies(df["Country"])
df = pd.concat([df, oh_c], axis=1)

### y값 예측 모델링

In [69]:
# y 변수 (HAND WARMER 포함 여부) 추가
df["temp"] = df["Description"].str.extract("(HAND WARMER)")

In [71]:
label_encoder.fit(df["temp"])
labels = label_encoder.transform(df["temp"])
temp = pd.DataFrame(data=labels, columns=["Y"])
temp["Y"].replace({0:1, 1:0}, inplace=True)
df["Y"] = temp["Y"]

In [74]:
# 필요 없는 열 제거
df.drop(columns=["index","temp"], inplace=True)

In [98]:
df1 = df.select_dtypes(include=[np.number])

In [99]:
df1 = df1.astype("int")

In [100]:
df1

,Quantity,UnitPrice,Packed,Numbered,Set,Box,Amp,Plus,Col_op,Des_op,...,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified,Y
0,6,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,6,3,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,6,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,3,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540450,12,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
540451,6,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
540452,4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
540453,4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
df1.isna().sum().sum()

0

In [102]:
y_df1 = df1['Y']
X_df1 = df1.drop('Y',axis=1)

In [146]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df1, y_df1, test_size = 0.2)

In [147]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [153]:
dt_clf = DecisionTreeClassifier(random_state=111)
rf_clf = RandomForestClassifier(random_state=111)
lr_clf = LogisticRegression(solver ='liblinear', max_iter=1000)

In [154]:
dt_clf.fit(X_train.values, y_train)
dt_pred = dt_clf.predict(X_test.values)
print('정확도:{0:.8f}'.format(accuracy_score(y_test, dt_pred)))

rf_clf.fit(X_train.values, y_train)
rf_pred = rf_clf.predict(X_test.values)
print('정확도:{0:.8f}'.format(accuracy_score(y_test, rf_pred)))

lr_clf.fit(X_train.values, y_train)
lr_pred = lr_clf.predict(X_test.values)
print('정확도:{0:.8f}'.format(accuracy_score(y_test, lr_pred)))

정확도:0.99480068
정확도:0.99481918
정확도:0.99281161
